## Trường Đại học Khoa học Tự nhiên - Đại học Quốc gia TP.HCM

# **Khoa Công nghệ thông tin**

## Lớp: **Lập trình song song - 21KHMT**

# Đồ án cuối kỳ: **Implement and optimize the traditional Artificial Neural Network (ANN)**

---

**Sinh viên thực hiện:** Nhóm 05 - Thần y xứ Quảng

| STT | MSSV     | Họ và tên         |
| :-: | -------- | ----------------- |
|  1  | 21127084 | Lê Xuân Kiên      |
|  2  | 21127108 | Đặng Hà Nhật Minh |
|  3  | 21127739 | Vũ Minh Phát      |

**Giảng viên hướng dẫn:**

- Nguyễn Trần Duy Minh
- Phạm Trọng Nghĩa
- Nguyễn Thanh Tình

---


# 0. Cài đặt môi trường

Cài đặt Nsight System trên Google Colab:

In [2]:
# Here is the command to install Nsight system on Colab. You just need to run the cell.
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
!apt update
!apt install ./nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
!apt --fix-broken install

--2024-12-26 07:48:28--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.199.39.144
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.199.39.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 317705436 (303M) [application/x-deb]
Saving to: ‘nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb.1’

nsight-systems-2023 100%[===================>] 302.99M   280MB/s    in 1.1s    

2024-12-26 07:48:29 (280 MB/s) - ‘nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb.1’ saved [317705436/317705436]

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 ht

#  
---

# 1. Host Model V1

In [3]:
%%time
!nvcc -O3 -use_fast_math host_v1.cu -diag-suppress 177 -o host_v1

CPU times: user 31.9 ms, sys: 6.11 ms, total: 38 ms
Wall time: 4.62 s


In [4]:
%%time
!./host_v1 # ~ 3min

Loading Fashion-MNIST dataset...
Training the Neural Network using mini-batch gradient descent...
==================+=================+========================
[ NUM_EPOCHS =  5 | BATCH_SIZE = 64 | LEARNING_RATE = 0.100 ]
------------------+-----------------+------------------------
[>] Epoch 1/5, Train loss: 0.4319695234, Train accuracy: 0.841767
[>] Epoch 2/5, Train loss: 0.3727906644, Train accuracy: 0.863367
[>] Epoch 3/5, Train loss: 0.3395258188, Train accuracy: 0.875933
[>] Epoch 4/5, Train loss: 0.3253435791, Train accuracy: 0.877867
[>] Epoch 5/5, Train loss: 0.3152741492, Train accuracy: 0.883317

Evaluating the model on the full test set...
[>] Test loss: 0.3837175369
[>] Test accuracy: 0.862400

[========== MODEL RUN TIME REPORT ==========]
(*) Average time for forward and backward pass of each layer in 1 epoch
+----------+--------------+---------------+
| Layer    | Forward (ms) | Backward (ms) |
|==========+==============+===============|
| Hidden 1 |     7906.064 |      

#  
---

# 2. Host Model V2

In [5]:
%%time
!nvcc -O3 -use_fast_math host_v2.cu -diag-suppress 177 -o host_v2

CPU times: user 23.8 ms, sys: 4.86 ms, total: 28.7 ms
Wall time: 3.52 s


In [6]:
%%time
!./host_v2 # ~ 3min

Loading Fashion-MNIST dataset...
Training the Neural Network using mini-batch gradient descent...
==================+=================+========================
[ NUM_EPOCHS =  5 | BATCH_SIZE = 64 | LEARNING_RATE = 0.100 ]
------------------+-----------------+------------------------
[>] Epoch   1/5, Train loss: 0.4434850812, Train accuracy: 0.834967
[>] Epoch   2/5, Train loss: 0.3818791807, Train accuracy: 0.860283
[>] Epoch   3/5, Train loss: 0.3441822827, Train accuracy: 0.874333
[>] Epoch   4/5, Train loss: 0.3216446042, Train accuracy: 0.880567
[>] Epoch   5/5, Train loss: 0.3372273743, Train accuracy: 0.872683

Evaluating the model on the full test set...
[>] Test loss: 0.4033925533
[>] Test accuracy: 0.852400

[========== MODEL RUN TIME REPORT ==========]
(*) Average time for forward and backward pass of each layer in 1 epoch
+----------+--------------+---------------+
| Layer    | Forward (ms) | Backward (ms) |
|==========+==============+===============|
| Hidden 1 |     8345.4

#  
---

# 3. Parallel Model V1

In [7]:
%%time
!nvcc -O3 -use_fast_math parallel_v1_GMEM.cu -diag-suppress 177 -o parallel_v1_GMEM

CPU times: user 25.7 ms, sys: 3.9 ms, total: 29.6 ms
Wall time: 3.62 s


In [8]:
%%time
!nsys nvprof ./parallel_v1_GMEM # ~ 2min


**********GPU Info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15835660288 bytes
CMEM: 65536 bytes
L2 cache: 4194304 bytes
SMEM / one SM: 65536 bytes
****************************

Loading Fashion-MNIST dataset...
Training the Neural Network using mini-batch gradient descent...
==================+=================+========================
[ NUM_EPOCHS = 10 | BATCH_SIZE = 64 | LEARNING_RATE = 0.100 ]
------------------+-----------------+------------------------
[>] Epoch   1/10, Train loss: 0.4451098740, Train accuracy: 0.834117
[>] Epoch   2/10, Train loss: 0.3814660609, Train accuracy: 0.859900
[>] Epoch   3/10, Train loss: 0.3448198438, Train accuracy: 0.873317
[>] Epoch   4/10, Train loss: 0.3181759119, Train accuracy: 0.880867
[>] Epoch   5/10, Train loss: 0.3298675716, Train accuracy: 0.874233
[>] Epoch   6/10, Train loss: 0.2879684567, Train accuracy: 0.891150
[>] Epoch   7/10, Train loss: 0.2725281417, 

#  
---

# 4. Parallel Model V2

In [9]:
%%time
!nvcc -O3 -use_fast_math parallel_v2_optimized.cu -diag-suppress 177 -o parallel_v2_optimized

CPU times: user 30.2 ms, sys: 2.71 ms, total: 32.9 ms
Wall time: 3.72 s


In [10]:
%%time
!nsys nvprof ./parallel_v2_optimized # ~ 2min


**********GPU Info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15835660288 bytes
CMEM: 65536 bytes
L2 cache: 4194304 bytes
SMEM / one SM: 65536 bytes
****************************

Loading Fashion-MNIST dataset...
Training the Neural Network using mini-batch gradient descent...
==================+=================+========================
[ NUM_EPOCHS = 10 | BATCH_SIZE = 64 | LEARNING_RATE = 0.100 ]
------------------+-----------------+------------------------
[>] Epoch   1/10, Train loss: 0.4451112747, Train accuracy: 0.834117
[>] Epoch   2/10, Train loss: 0.3814668655, Train accuracy: 0.859900
[>] Epoch   3/10, Train loss: 0.3448202610, Train accuracy: 0.873317
[>] Epoch   4/10, Train loss: 0.3181747794, Train accuracy: 0.880867
[>] Epoch   5/10, Train loss: 0.3298676312, Train accuracy: 0.874233
[>] Epoch   6/10, Train loss: 0.2879679501, Train accuracy: 0.891150
[>] Epoch   7/10, Train loss: 0.2725301683, 

#  
---

# 5. Parallel Model V3

In [11]:
%%time
!nvcc -O3 -use_fast_math parallel_v3_optimized.cu -diag-suppress 177 -o parallel_v3_optimized

CPU times: user 31.3 ms, sys: 0 ns, total: 31.3 ms
Wall time: 3.72 s


In [12]:
%%time
!nsys nvprof ./parallel_v3_optimized # ~ 1min


**********GPU Info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15835660288 bytes
CMEM: 65536 bytes
L2 cache: 4194304 bytes
SMEM / one SM: 65536 bytes
****************************

Loading Fashion-MNIST dataset...
[>] Dataset loaded in 1016.737 ms
[>] Data prepared in 1.766 ms
Training the Neural Network using mini-batch gradient descent...
==================+=================+========================
[ NUM_EPOCHS = 10 | BATCH_SIZE = 64 | LEARNING_RATE = 0.100 ]
------------------+-----------------+------------------------
[>] Epoch   1/10, Train loss: 0.4174713194, Train accuracy: 0.845867
[>] Epoch   2/10, Train loss: 0.3641712368, Train accuracy: 0.865933
[>] Epoch   3/10, Train loss: 0.3314569592, Train accuracy: 0.877183
[>] Epoch   4/10, Train loss: 0.3096896410, Train accuracy: 0.883950
[>] Epoch   5/10, Train loss: 0.3213946819, Train accuracy: 0.880967
[>] Epoch   6/10, Train loss: 0.2815082073, Train

#  
---

# 6. Parallel Model V4

In [13]:
%%time
!nvcc -O3 -use_fast_math parallel_v4_optimized.cu -diag-suppress 177 -o parallel_v4_optimized

CPU times: user 28.6 ms, sys: 4.23 ms, total: 32.8 ms
Wall time: 4.02 s


In [14]:
%%time
!nsys nvprof ./parallel_v4_optimized # ~ 30s


**********GPU Info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15835660288 bytes
CMEM: 65536 bytes
L2 cache: 4194304 bytes
SMEM / one SM: 65536 bytes
****************************

Loading Fashion-MNIST dataset...
[>] Dataset loaded in 1030.103 ms
[>] Data prepared in 1.866 ms
Training the Neural Network using mini-batch gradient descent...
==================+=================+========================
[ NUM_EPOCHS = 10 | BATCH_SIZE = 64 | LEARNING_RATE = 0.100 ]
------------------+-----------------+------------------------
[>] Weight initialization time: 2.837184 ms
[>] Epoch   1/10, Train loss: 0.4174720645, Train accuracy: 0.845867
[>] Epoch   2/10, Train loss: 0.3641698658, Train accuracy: 0.865933
[>] Epoch   3/10, Train loss: 0.3314563334, Train accuracy: 0.877183
[>] Epoch   4/10, Train loss: 0.3096899986, Train accuracy: 0.883950
[>] Epoch   5/10, Train loss: 0.3213953972, Train accuracy: 0.880967
[>] E